In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from ase.io import read
from ase.visualize.plot import plot_atoms
from mpl_toolkits.mplot3d import Axes3D
import random
import numpy as np
import os
import pandas as pd
import numpy as np
from ase.io import read
from tqdm import tqdm


In [7]:
# Path setup
data_dir = "./"  # <-- Change this to your actual data path
atoms_train_dir = os.path.join(data_dir, "atoms", "train")
energies_csv = os.path.join(data_dir, "energies", "train.csv")

In [19]:
# Read energies CSV
energies_df = pd.read_csv(energies_csv)
print("Loaded energy data:\n", energies_df.head().to_string(index=False))

Loaded energy data:
  id     energy
  1 -90.107880
  2 -69.927647
  3 -69.979214
  4 -73.402302
  5 -78.936724


In [20]:
# Analyze number of atoms per molecule
num_atoms_list = []
for file in sorted(os.listdir(atoms_train_dir)):
    if file.endswith(".xyz"):
        atoms = read(os.path.join(atoms_train_dir, file))
        num_atoms_list.append(len(atoms))

In [20]:
# # Plot distribution of number of atoms
# plt.figure(figsize=(8, 5))
# plt.hist(num_atoms_list, bins=20, color='skyblue', edgecolor='black')
# plt.title("Distribution of Number of Atoms per Molecule")
# plt.xlabel("Number of atoms")
# plt.ylabel("Count")
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [21]:
# # Plot distribution of energies
# plt.figure(figsize=(8, 5))
# plt.hist(energies_df['energy'], bins=30, color='lightgreen', edgecolor='black')
# plt.title("Distribution of Atomization Energies")
# plt.xlabel("Energy (a.u.)")
# plt.ylabel("Count")
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [22]:
# sample_files = random.sample(sorted(os.listdir(atoms_train_dir)), 3)
# for fname in sample_files:
#     atoms = read(os.path.join(atoms_train_dir, fname))
#     fig, ax = plt.subplots(figsize=(5, 5))
#     plot_atoms(atoms, ax, radii=0.3)
#     ax.set_title(f"Molecule: {fname}")
#     plt.axis('off')
#     plt.show()

In [12]:
# === Extraction des features depuis les fichiers .xyz ===
def extract_features(xyz_path):
    atoms = read(xyz_path)
    masses = atoms.get_masses()
    charges = atoms.get_atomic_numbers()
    com = atoms.get_center_of_mass()
    max_dist = np.linalg.norm(atoms.positions - com, axis=1).max()
    
    return {
        "nb_atoms": len(atoms),
        "masse_totale": masses.sum(),
        "Z_moyen": charges.mean(),
        "dist_centre_max": max_dist
    }

In [21]:
# Get all files and extract IDs
files = [f for f in os.listdir(atoms_train_dir) if f.endswith(".xyz")]

# Extract IDs and sort numerically
files.sort(key=lambda x: int(x.split("_")[1].split(".")[0]))

data = []
for fname in tqdm(files):
    mol_id = int(fname.split("_")[1].split(".")[0])
    xyz_path = os.path.join(atoms_train_dir, fname)
    feats = extract_features(xyz_path)
    feats["id"] = mol_id  # Use the extracted ID
    data.append(feats)

features_df = pd.DataFrame(data)

  0%|          | 0/6591 [00:00<?, ?it/s]

100%|██████████| 6591/6591 [00:03<00:00, 1837.73it/s]


In [22]:
features_df.head()

,nb_atoms,masse_totale,Z_moyen,dist_centre_max,id
0,20,99.177,2.800,4.426248,1
1,15,87.122,3.200,3.174427,2
2,15,87.122,3.200,3.248984,3
3,16,90.122,3.125,2.853333,4
4,16,95.145,3.250,3.135213,5


In [23]:
merged_df = pd.merge(features_df, energies_df, on="id")[['id', 'nb_atoms', 'masse_totale', 'Z_moyen', 'dist_centre_max', 'energy']]
print(merged_df.head())

   id  nb_atoms  masse_totale  Z_moyen  dist_centre_max     energy
0   1        20        99.177    2.800         4.426248 -90.107880
1   2        15        87.122    3.200         3.174427 -69.927647
2   3        15        87.122    3.200         3.248984 -69.979214
3   4        16        90.122    3.125         2.853333 -73.402302
4   5        16        95.145    3.250         3.135213 -78.936724


In [ ]:
# # Load both
# pred_df = pd.read_csv(pred_csv)

# # Merge by id to ensure alignment
# merged = pd.merge(energies_df, pred_df, on="id", suffixes=("_true", "_pred"))

# # Compute RMSE
# rmse = np.sqrt(np.mean((merged["energy_true"] - merged["energy_pred"]) ** 2))
# print(f"RMSE: {rmse:.6f}")